In [1]:
from sklearn.ensemble import RandomForestClassifier

def create_rf(n_trees):
    return RandomForestClassifier(n_estimators=n_trees)

Q:\ProgramData\Miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
Q:\ProgramData\Miniconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
Q:\ProgramData\Miniconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
import pandas as pd
import numpy as np

In [3]:
ids = pd.read_csv('csv/ids.csv')
ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  55 non-null     int64 
 1   number      55 non-null     object
 2   ill         55 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ KB


In [4]:
names = ['all', 'night', 'day']
selected = {}
hourly_selected = {}

for n in names:
    selected[n] = pd.read_csv(f'csv/selected_{n}.csv')
    hourly_selected[n] = pd.read_csv(f'csv/selected_hourly_{n}.csv')

In [5]:
from sklearn.model_selection import cross_val_score

folds = 10
y = ids['ill']

def cross_validate(name: str, data: pd.DataFrame):
    print(f'{folds}-fold cross validation on dataset: {name}')
    
    rf500 = create_rf(500)
    rf1000 = create_rf(1000)
    
    scores500 = cross_val_score(rf500, data, y, cv=folds)
    print(f'Average score on 500 trees RF: {np.mean(scores500)}')
    print('All scores:', scores500)
    
    scores1000 = cross_val_score(rf1000, data, y, cv=folds)
    print(f'Average score on 1000 trees RF: {np.mean(scores1000)}')
    print('All scores:', scores1000)

In [6]:
for name, data in selected.items():
    cross_validate(name, data)

10-fold cross validation on dataset: all
Average score on 500 trees RF: 0.9233333333333335
All scores: [0.83333333 1.         1.         1.         1.         1.
 1.         0.8        0.8        0.8       ]
Average score on 1000 trees RF: 0.9433333333333336
All scores: [0.83333333 1.         1.         1.         1.         1.
 1.         1.         0.8        0.8       ]
10-fold cross validation on dataset: night
Average score on 500 trees RF: 0.8333333333333334
All scores: [1.         1.         0.5        0.83333333 1.         0.8
 1.         0.6        0.6        1.        ]
Average score on 1000 trees RF: 0.8533333333333333
All scores: [1.         0.83333333 0.66666667 0.83333333 1.         0.8
 1.         0.8        0.6        1.        ]
10-fold cross validation on dataset: day
Average score on 500 trees RF: 0.9633333333333335
All scores: [0.83333333 1.         1.         1.         1.         1.
 1.         1.         1.         0.8       ]
Average score on 1000 trees RF: 0.96

In [7]:
for name, data in hourly_selected.items():
    cross_validate(name, data)

10-fold cross validation on dataset: all
Average score on 500 trees RF: 0.9800000000000001
All scores: [1.  1.  1.  1.  1.  1.  1.  1.  1.  0.8]
Average score on 1000 trees RF: 0.9433333333333334
All scores: [0.83333333 1.         1.         1.         1.         1.
 1.         0.8        1.         0.8       ]
10-fold cross validation on dataset: night
Average score on 500 trees RF: 0.9066666666666666
All scores: [1.         1.         0.83333333 0.83333333 1.         0.8
 1.         0.6        1.         1.        ]
Average score on 1000 trees RF: 0.9266666666666667
All scores: [1.         1.         0.83333333 0.83333333 1.         1.
 1.         0.6        1.         1.        ]
10-fold cross validation on dataset: day
Average score on 500 trees RF: 0.9633333333333335
All scores: [0.83333333 1.         1.         1.         1.         1.
 1.         1.         1.         0.8       ]
Average score on 1000 trees RF: 0.9633333333333335
All scores: [0.83333333 1.         1.         1. 